In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
#jobdescription is tokenized into description
import pandas as pd
import nltk

df = pd.read_csv("/content/sample_data/dice_com-job_us_sample.csv")


def identify_tokens(row):
    jobdescription = row['jobdescription']
    tokens = nltk.word_tokenize(jobdescription)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

df['description'] = df.apply(identify_tokens, axis=1)
df

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id,description
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4,"[Looking, for, Selenium, engineers, must, have..."
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9,"[The, University, of, Chicago, has, a, rapidly..."
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72,"[day, our, solutions, affect, people, througho..."
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,Please see job description,3941b2f206ae0f900c4fba4ac0b18719,"[Java, IL, Our, client, is, a, leading, edge, ..."
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"Configuration Management, Developer, Linux, Ma...",45efa1f6bc65acc32bbbb953a1ed13b7,"[Midtown, based, high, tech, firm, has, an, im..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,https://www.dice.com/jobs/detail/Web-Designer-...,IAC Publishing,Full Time,Company Description We are searching for a ta...,Dice Id : 10112803,"Oakland, CA",Web Designer,3 weeks ago,Telecommuting not available|Travel not required,NaN,"UI/UX mobile apps, interaction design, digital...",86e27ce6b7e631e55d69d142c7d43df2,"[Company, Description, We, are, searching, for..."
21996,https://www.dice.com/jobs/detail/Senior-Front-...,Omega Solutions Inc,Full Time,CONTACT - priya@omegasolutioninc.com / 408-45...,Dice Id : 10289500,"San Francisco, CA",Senior Front End Web Developer - Full Time at ...,3 weeks ago,Telecommuting not available|Travel not required,NaN,"JavaScript, HTML5, CSS3, Bootstrap, AJAX, Reac...",4287c7ee3317ccf1edd76e238cf8e584,"[CONTACT, priya, San, Francisco, CATerm, Full,..."
21997,https://www.dice.com/jobs/detail/QA-Analyst-Sa...,San Francisco Health Plan,Full Time,Do you take pride in your work knowing that th...,Dice Id : 10115761,"San Francisco, CA",QA Analyst,2 weeks ago,Telecommuting not available|Travel not required,NaN,"SDLC, ALM, SQL, T-SQL, RedGate, Team Foundatio...",d7512f0181d69f83f96db38cd77a4d08,"[Do, you, take, pride, in, your, work, knowing..."
21998,https://www.dice.com/jobs/detail/Tech-Lead%252...,IAC Publishing,Full Time,Company Description What We Can Offer YouAs th...,Dice Id : 10112803,"Oakland, CA",Tech Lead-Full Stack,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Python, Ruby, Go, Clojure, Java, NoSQL-Databas...",ec375268b494b3bcbed1635d64226112,"[Company, Description, What, We, Can, Offer, Y..."


In [4]:
# stopwords are removed from description and stored into new column named meaningful
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['description']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

df['meaningful'] = df.apply(remove_stops, axis=1)
df

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id,description,meaningful
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4,"[Looking, for, Selenium, engineers, must, have...","[Looking, Selenium, engineers, must, solid, ja..."
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9,"[The, University, of, Chicago, has, a, rapidly...","[The, University, Chicago, rapidly, growing, s..."
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72,"[day, our, solutions, affect, people, througho...","[day, solutions, affect, people, throughout, w..."
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,Please see job description,3941b2f206ae0f900c4fba4ac0b18719,"[Java, IL, Our, client, is, a, leading, edge, ...","[Java, IL, Our, client, leading, edge, softwar..."
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"Configuration Management, Developer, Linux, Ma...",45efa1f6bc65acc32bbbb953a1ed13b7,"[Midtown, based, high, tech, firm, has, an, im...","[Midtown, based, high, tech, firm, immediate, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,https://www.dice.com/jobs/detail/Web-Designer-...,IAC Publishing,Full Time,Company Description We are searching for a ta...,Dice Id : 10112803,"Oakland, CA",Web Designer,3 weeks ago,Telecommuting not available|Travel not required,NaN,"UI/UX mobile apps, interaction design, digital...",86e27ce6b7e631e55d69d142c7d43df2,"[Company, Description, We, are, searching, for...","[Company, Description, We, searching, talented..."
21996,https://www.dice.com/jobs/detail/Senior-Front-...,Omega Solutions Inc,Full Time,CONTACT - priya@omegasolutioninc.com / 408-45...,Dice Id : 10289500,"San Francisco, CA",Senior Front End Web Developer - Full Time at ...,3 weeks ago,Telecommuting not available|Travel not required,NaN,"JavaScript, HTML5, CSS3, Bootstrap, AJAX, Reac...",4287c7ee3317ccf1edd76e238cf8e584,"[CONTACT, priya, San, Francisco, CATerm, Full,...","[CONTACT, priya, San, Francisco, CATerm, Full,..."
21997,https://www.dice.com/jobs/detail/QA-Analyst-Sa...,San Francisco Health Plan,Full Time,Do you take pride in your work knowing that th...,Dice Id : 10115761,"San Francisco, CA",QA Analyst,2 weeks ago,Telecommuting not available|Travel not required,NaN,"SDLC, ALM, SQL, T-SQL, RedGate, Team Foundatio...",d7512f0181d69f83f96db38cd77a4d08,"[Do, you, take, pride, in, your, work, knowing...","[Do, take, pride, work, knowing, thousands, li..."
21998,https://www.dice.com/jobs/detail/Tech-Lead%252...,IAC Publishing,Full Time,Company Description What We Can Offer YouAs th...,Dice Id : 10112803,"Oakland, CA",Tech Lead-Ful

In [5]:
#meaningful is again detokenized into meaninful string

import pandas as pd
from nltk.tokenize.treebank import TreebankWordDetokenizer

df['meaningfulstring']=df['meaningful'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))

df['meaningfulstring']=df['meaningfulstring'].str.lower()
df['skills']=df['skills'].str.lower()
df['jobtitle']=df['jobtitle'].str.lower()
df['skills']=df['skills'].fillna(' ')

# see below and see job description strings are removed from skills
for i in df.index:
  if(df['skills'].iloc[i]=="see below" or df['skills'].iloc[i]=="(see job description)" ):
    df['skills'].iloc[i]=" "

#skills and meaningfulstring is combined into one column named combined    
def parameter(row):    
    return row['skills']+" "+row['meaningfulstring']

df['combined']=df.apply(parameter,axis=1)    
df['combined'].head(3)


0      looking selenium engineers must solid java c...
1    linux/unix, network monitoring, incident respo...
2    enterprise solutions architecture, business in...
Name: combined, dtype: object

In [6]:
#job id column is inserted to identify each tuple in the next step
df.insert(0,'Job id',range(1,1+len(df)))
df.head(3)

,Job id,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id,description,meaningful,meaningfulstring,combined
0,1,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",automation test engineer,1 hour ago,Telecommuting not available|Travel not required,NaN,,418ff92580b270ef4e7c14f0ddfc36b4,"[Looking, for, Selenium, engineers, must, have...","[Looking, Selenium, engineers, must, solid, ja...",looking selenium engineers must solid java cod...,looking selenium engineers must solid java c...
1,2,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",information security engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9,"[The, University, of, Chicago, has, a, rapidly...","[The, University, Chicago, rapidly, growing, s...",the university chicago rapidly growing securit...,"linux/unix, network monitoring, incident respo..."
2,3,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",business solutions architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"enterprise solutions architecture, business in...",46baa1f69ac07779274bcd90b85d9a72,"[day, our, solutions, affect, people, througho...","[day, solutions, affect, people, throughout, w...",day solutions affect people throughout world f...,"enterprise solutions architecture, business in..."


In [8]:
df=df.drop(['jobid','uniq_id','description', 'meaningful','meaningfulstring'], axis = 1)
df.head(3)

,Job id,advertiserurl,company,employmenttype_jobstatus,jobdescription,joblocation_address,jobtitle,postdate,shift,site_name,skills,combined
0,1,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,"Atlanta, GA",automation test engineer,1 hour ago,Telecommuting not available|Travel not required,NaN,,looking selenium engineers must solid java c...
1,2,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,"Chicago, IL",information security engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...","linux/unix, network monitoring, incident respo..."
2,3,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...","Schaumburg, IL",business solutions architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"enterprise solutions architecture, business in...","enterprise solutions architecture, business in..."


In [10]:
df.to_csv('Preprocessed.csv')